In [ ]:
import json
import os
import numpy as np
import re
import pandas as pd
from rdkit import Chem
from rdkit.Chem import rdchem, Descriptors, rdMolDescriptors
import numpy as np
from ase.units import Hartree, eV, Bohr, Ang

prop_names = ['rcA', 'rcB', 'rcC', 'mu', 'alpha', 'homo', 'lumo',
                  'gap', 'r2', 'zpve', 'energy_U0', 'energy_U', 'enthalpy_H',
                  'free_G', 'Cv']
conversions = [1., 1., 1., 1., Bohr ** 3 / Ang ** 3,
                Hartree / eV, Hartree / eV, Hartree / eV,
                Bohr ** 2 / Ang ** 2, Hartree / eV,
                Hartree / eV, Hartree / eV, Hartree / eV,
                Hartree / eV, 1.]

In [ ]:
def from_xyz_to_json(file):
    mol_dict = {}

    with open(file) as f:
        data = f.readlines()

    smiles = data[-2].split()[-1]
    mol_dict["smiles"] = smiles

    mol_properties = data[1].split()[1:16]
    for i,prop in enumerate(mol_properties):
        mol_dict[prop_names[i]] = float(prop.replace("^", "")) * conversions[i]

    num_atoms = int(data[0])
    atoms = []
    for i, atom_info in enumerate(data[2:2 + num_atoms]):
        atom_split = atom_info.split()
        atom = (
            atom_split[0],
            float(atom_split[1].replace("^", "")),
            float(atom_split[2].replace("^", "")),
            float(atom_split[3].replace("^", ""))
        )
        atoms.append(atom)

    mol_dict["atoms"] = atoms
    return mol_dict

In [ ]:
input_folder = "data/133660_curatedQM9_outof_133885"
output_folder = "data/qm9_data_json"
os.makedirs(output_folder, exist_ok=True)

for i, mol_path in enumerate(sorted(os.listdir(input_folder))[137:]):
    path = os.path.join(input_folder, mol_path)
    mol_dict = from_xyz_to_json(path)
    
    json_path = f'qm9_{str(i+1).zfill(6)}.json'
    with open(os.path.join(output_folder, json_path), 'w') as json_file:
        json.dump(mol_dict, json_file)

    if i % 1000 == 0: 
        print(i)

In [ ]:
smiles_list = []
for i, file in enumerate(os.listdir(output_folder)):
    mol_path = os.path.join(output_folder, file)
    with open(mol_path, 'r') as f:
        mol = json.load(f)
    smiles_list.append(mol["smiles"])
    if i % 1000 == 0: print(i)

In [ ]:
# TEST FOR ATOMIC_NUMBER

atom_options = ["H", "C", "N", "O", "F"]

for sm in smiles_list:
    sm = ''.join(re.findall(r'[A-Z][a-z]?', sm))

    for atom in atom_options:
        sm = sm.replace(atom, "")
        sm = sm.replace(atom.lower(), "")
    
    if sm != "": 
        print(sm)

In [ ]:
min_num_bonds = 100
max_num_bonds = 0

min_hybridization = 100
max_hybridization = 0

min_aromatic = 100
max_aromatic = 0

min_num_heavy_neighbors = 100
max_num_heavy_neighbors = 0

min_chirality = 100
max_chirality = 0

min_valence = 100
max_valence = 0

min_in_ring = 100
max_in_ring = 0

error_count = 0

for smiles in smiles_list:
    mol = Chem.MolFromSmiles(smiles)
    if not mol:
        error_count += 1
        continue

    for atom in mol.GetAtoms():
        num_bonds = len(atom.GetBonds())
        hybridization = int(atom.GetHybridization())
        aromatic = int(atom.GetIsAromatic())
        chirality = int(atom.GetChiralTag())
        valence = atom.GetTotalValence()
        in_ring = int(atom.IsInRing())

        min_num_bonds = min(min_num_bonds, num_bonds)
        max_num_bonds = max(max_num_bonds, num_bonds)

        min_hybridization = min(min_hybridization, hybridization)
        max_hybridization = max(max_hybridization, hybridization)

        min_aromatic = min(min_aromatic, aromatic)
        max_aromatic = max(max_aromatic, aromatic)

        min_chirality = min(min_chirality, chirality)
        max_chirality = max(max_chirality, chirality)

        min_valence = min(min_valence, valence)
        max_valence = max(max_valence, valence)

        min_in_ring = min(min_in_ring, in_ring)
        max_in_ring = max(max_in_ring, in_ring)

print(error_count)

In [ ]:
print(f"  num_bonds: (min: {min_num_bonds}, max: {max_num_bonds})")
print(f"  hybridization: (min: {min_hybridization}, max: {max_hybridization})")
print(f"  aromatic: (min: {min_aromatic}, max: {max_aromatic})")
print(f"  chirality: (min: {min_chirality}, max: {max_chirality})")
print(f"  valence: (min: {min_valence}, max: {max_valence})")
print(f"  in_ring: (min: {min_in_ring}, max: {max_in_ring})")

In [ ]:
def get_edge_features(mol):
    """Extracts bond features, including distances and angles."""
    edges, distances = [], []
    axes = {"X": np.array([1, 0, 0]), "Y": np.array([0, 1, 0]), "Z": np.array([0, 0, 1])}

    conf = mol.GetConformer()

    for bond in mol.GetBonds():
        i, j = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
        pos_i, pos_j = np.array(conf.GetAtomPosition(i)), np.array(conf.GetAtomPosition(j))
        vector_ij, norm_vector_ij = pos_j - pos_i, np.linalg.norm(pos_j - pos_i)
        
        if norm_vector_ij == 0:
            continue  # Avoid division by zero
        
        distances.append(norm_vector_ij)
        angles = {f"angle_{axis}": np.degrees(np.arccos(np.clip(np.dot(vector_ij, axis_vec) / norm_vector_ij, -1.0, 1.0))) for axis, axis_vec in axes.items()}
        
        bond_order = {
            rdchem.BondType.SINGLE: 0.33,
            rdchem.BondType.DOUBLE: 0.66,
            rdchem.BondType.TRIPLE: 1.0,
            rdchem.BondType.AROMATIC: 0.5
        }.get(bond.GetBondType(), 0)
        
        edges.append([i, j, bond_order, norm_vector_ij, angles["angle_X"], angles["angle_Y"], angles["angle_Z"]])
    
    edges = np.array(edges)
    edges[:, 4:] /= 180  # Normalize angles
    
    return edges

def add_conformer_to_mol(mol, df):
    conf = Chem.Conformer(mol.GetNumAtoms())

    for i, row in df.iterrows():
        conf.SetAtomPosition(i, (row['x'], row['y'], row['z']))
    
    mol.AddConformer(conf, assignId=True)
    return mol

In [ ]:
with open("data/means_and_stds.json") as f:
    scaler = json.load(f)

def normalize_df(df, scaler):
    df["x"] = (df["x"] - scaler["x_mean"]) / scaler["x_std"]
    df["y"] = (df["y"] - scaler["y_mean"]) / scaler["y_std"]
    df["z"] = (df["z"] - scaler["z_mean"]) / scaler["z_std"]
    return df

def preprocess_df(df, positions):
    """Preprocesses atomic and positional data for a molecule."""
    
    # Define atomic number mappings
    atom_options = {6: "C", 7: "N", 8: "O", 9: "F"}
    
    # One-hot encoding for atomic numbers
    for key, label in atom_options.items():
        df[label] = (df["atomic_num"] == key).astype(int)
    df.drop(columns=["atomic_num"], inplace=True)
    
    # Normalize features
    features = {"num_bonds": 4, "hybridization": 4, "aromatic": 1, "chirality": 2, "valence": 4, "in_ring": 1}
    for feature, val in features.items():
        df[feature] = df[feature].clip(0, val) / val
        
    # Process positional data
    positions = normalize_df(positions, scaler).reset_index()
    positions_df = pd.DataFrame(positions, columns=["x", "y", "z"])
    df = df.join(positions_df)
    
    return df, positions_df

def get_atom_features(atom):
    """Extracts atomic features from an RDKit atom object."""
    return {
        "atomic_num": atom.GetAtomicNum(),
        "num_bonds": len(atom.GetBonds()),
        "hybridization": int(atom.GetHybridization()),
        "aromatic": int(atom.GetIsAromatic()),
        "chirality": int(atom.GetChiralTag()),
        "valence": atom.GetTotalValence(),
        "in_ring": int(atom.IsInRing()),
    }

In [ ]:
def calculate_descriptors(smiles, mol, homo, lumo):
 
    desc = {
        "SMILES": smiles,
        "MolWt": Descriptors.MolWt(mol),  # Masa molowa
        "LogP": Descriptors.MolLogP(mol),  # LogP (lipofilowość)
        "TPSA": Descriptors.TPSA(mol),  # Topologiczna powierzchnia polarowa
        "HDonors": Descriptors.NumHDonors(mol),  # Liczba donorów wiązań wodorowych
        "HAcceptors": Descriptors.NumHAcceptors(mol),  # Liczba akceptorów wiązań wodorowych
        "RotatableBonds": Descriptors.NumRotatableBonds(mol),  # Liczba rotowalnych wiązań
        "RingCount": rdMolDescriptors.CalcNumRings(mol),  # Liczba pierścieni
        "Kappa1": Descriptors.Kappa1(mol),  # Indeks kształtu Kappa 1
        "Kappa2": Descriptors.Kappa2(mol),  # Indeks kształtu Kappa 2
        "Kappa3": Descriptors.Kappa3(mol),  # Indeks kształtu Kappa 3
        "Polarizability": Descriptors.MolMR(mol),  # Polarowalność molowa
        "MaxEStateIndex": Descriptors.MaxEStateIndex(mol),  # Maksymalny indeks EState
        "MinEStateIndex": Descriptors.MinEStateIndex(mol),  # Minimalny indeks EState
        "CPSA1": rdMolDescriptors.CalcAUTOCORR3D(mol)[0] if len(rdMolDescriptors.CalcAUTOCORR3D(mol)) > 0 else None,
        "RDF": rdMolDescriptors.CalcRDF(mol)[0] if len(rdMolDescriptors.CalcRDF(mol)) > 0 else None,  # Deskryptor 3D RDF
        "HOMO": homo,  # Przybliżony wskaźnik QED jako zastępnik HOMO
        "LUMO": lumo,  # Przybliżony wskaźnik dla LUMO
        "Asphericity": rdMolDescriptors.CalcAsphericity(mol),  # Anizotropia kształtu
        "Eccentricity": rdMolDescriptors.CalcEccentricity(mol)  # Ekscentryczność cząsteczki
    }

    return desc

In [ ]:
def craete_pos_df(atoms):
    arr = []
    for i,atom in enumerate(atoms):
        if atom[0] != "H":
            res = {}
            res["x"] = atom[1]
            res["y"] = atom[2]
            res["z"] = atom[3]
            arr.append(res)
    return pd.DataFrame(arr)

def create_df_from_mol(smiles, atoms):
    mol = Chem.MolFromSmiles(smiles)
    node_features = [get_atom_features(atom) for atom in mol.GetAtoms()]
    atoms = craete_pos_df(atoms)
    nodes, positions = preprocess_df(pd.DataFrame(node_features), atoms)
    mol = add_conformer_to_mol(mol, positions)
    edges = get_edge_features(mol)
    return  mol, nodes, edges

In [ ]:
folder = "data/qm9_data"
os.makedirs(folder, exist_ok=True)

for idx, file in enumerate(os.listdir(output_folder)):
    path = os.path.join(output_folder, file)

    with open(path, 'r') as f:
        data = json.load(f)
 
    mol = Chem.MolFromSmiles(data["smiles"])
    if mol and len(mol.GetAtoms()) != 1:
        mol, nodes, edges = create_df_from_mol(data["smiles"], data["atoms"])
        final_path = os.path.join(folder, data["smiles"].replace("/", "").replace("\\", ""))
        os.makedirs(final_path, exist_ok=True)
        nodes.to_parquet(os.path.join(final_path, "nodes.parquet"))
        np.save(os.path.join(final_path, "edges"), edges)
        Chem.MolToMolFile(mol, os.path.join(final_path,"molecule.mol"))

        descriptor = calculate_descriptors(data["smiles"], mol, data["homo"], data["lumo"])
        descriptor["file_path"] = os.path.join(final_path, "molecule.mol")
        with open(os.path.join(final_path, "descriptor.json"), "w") as f:
            json.dump(descriptor, f)

    if idx % 1000 == 0:
        print(idx)